In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import itertools
from itertools import combinations

def main():
    todays_players = [40,6,23,26,15,98,1,10,55,16,
                      56,97,96,12,13,41,52,88,66,94,
                      89,86,87,77,79,72,76]
    input_data = pd.read_csv('MahjongScores.csv')

    pairings_df = create_pairings_table(todays_players)
    matchups_df = create_freq_matchups(pairings_df, input_data)
    
    table_info = get_table_info(todays_players)
    
    output_tables = generate_tables(table_info, matchups_df)
    
    print output_tables

In [2]:
def create_pairings_table(todays_players):
    # Create a zeros DataFrame of (#players,#players) size for matchups, with ordered indices.
    num_players = len(todays_players)
    pairings_df = DataFrame(np.zeros(num_players**2).reshape(num_players,num_players))
    pairings_df.index = sorted(todays_players)
    pairings_df.columns = sorted(todays_players)
    return pairings_df

In [3]:
def get_player_data(input_data, todays_players):
    # Reduce input_data to only the rows relevant to today's players
    games_rows = pd.DataFrame()
    for i in todays_players:
        games_rows = games_rows.append(input_data.loc[(input_data['PlayerId'] == i)])
    return games_rows

In [4]:
def create_freq_matchups(pairings_df, input_data):
    # Grab today's players and initialize output df
    todays_players = pairings_df.columns
    matchups_df = pairings_df.copy()
        
    # Pull all rows from input_data relevant to today's players
    games_rows = get_player_data(input_data, todays_players)
    
    # Make a list of unique games
    uniquegames = games_rows['GameId'].unique()
        
    # Get all players in each relevant game and populate matchup table one game at a time
    for gameid in uniquegames:
        gameid_players = []
        gameid_rows = games_rows.loc[(games_rows['GameId'] == gameid)] # Pull only the rows for players in that game
        gameid_players = gameid_rows['PlayerId'].tolist() #Give each player an index
        for i in range( len(gameid_players)): # For each player in a game
            for j in range( len(gameid_players)): 
                matchups_df.at[gameid_players[i],gameid_players[j]] += 1 # Add a game in opponent's column
    
    # Diagonal values are total # of games played for that player
    return matchups_df  

In [5]:
def create_score_matchups(pairings_df, input_data):
    todays_players = pairings_df.columns
    games_rows = get_player_data(input_data, todays_players)
    matchups_df = pairings_df.copy()
    
    for gameid in games_rows['GameId'].unique():
        # Pull rows for one game at a time
        gameid_players = []
        gameid_rows = games_rows.loc[(games_rows['GameId'] == gameid)]

        # Paired scores and players have the same index
        gameid_players = gameid_rows['PlayerId'].tolist()
        gameid_scores = gameid_rows['Score'].tolist()

        for i in range( len(gameid_players)): # For each player in that game
            flag = False
            for j in range( len(gameid_players)): # Get all players (including self)
                # Don't add along diagonal more than once for each player
                if (i == j):
                    if flag == False: 
                        matchups_df.at[gameid_players[i],gameid_players[j]] += gameid_scores[i]
                    flag = True
                else:
                    matchups_df.at[gameid_players[i],gameid_players[j]] += gameid_scores[i] # Add score of i to matchup sum
    
    freq_df = create_freq_matchups(pairings_df, input_data)
    for i in range( len(todays_players)):
        for j in range( len(todays_players)):
            matchups_df.iat[i,j] = matchups_df.iat[i,j] / np.diagonal(freq_df)[j]
    
    # Diagonal values are player's average score            
    return matchups_df

In [6]:
def get_table_counts(todays_players):
    # Determine amount of 5 person tables
    if len(todays_players) < 8:
        total_tables = 1
        if len(todays_players) == 4:
            tables_4p = 1
            tables_5p = 0
        elif len(todays_players) == 5:
            tables_4p = 0
            tables_5p = 1
        else:
            tables_4p = tables_5p = 0
    else:
        tables_5p = len(todays_players) % 4
        total_tables = int( len(todays_players) / 4)
        tables_4p = total_tables - tables_5p
    
    table_counts = [total_tables, tables_4p, tables_5p]
    
    return table_counts

In [7]:
def sum_ratings(table_players, matchups_df):
    table_sum = 0
    for i in table_players:
        for j in table_players:
            if i>j: # This only serves to prevent repeats
                table_sum += max(matchups_df.at[i,j],matchups_df.at[j,i])
    return table_sum

In [8]:
def generate_tables(table_types, matchups_df):
    output_tables = []
    total_tables = table_counts[0]
    tables_4p = table_counts[1]
    tables_5p = table_counts[2]
    tables_5p_made = 0
    temp_playerlist = list(matchups_df.columns) # Make a copy of today's players to modify
    
    # Make tables until all tables are made
    while len(output_tables) < total_tables:
    
        # Make final table if only 4 players are in pool
        if (len(temp_playerlist) == 4):
            output_tables.append(tuple(temp_playerlist))
            break
    
        # Generate list of combinations for table size
        if tables_5p_made < tables_5p:
            table_combs = list(itertools.combinations(temp_playerlist,5))
            tables_5p_made += 1
        else:
            table_combs = list(itertools.combinations(temp_playerlist,4))

        # Calculate matchmaking rating for each table (lower means players have fewer games together)
        table_df = DataFrame(table_combs)
        table_sums = table_df.apply(sum_ratings, args=[matchups_df], axis=1)    
       
        # Take first minimum value in table_sums - gives priority to lower PlayerId
        best_table_ix = np.argmin(table_sums)
        print table_sums[best_table_ix]
        output_tables.append(table_combs[best_table_ix])
    
        # Remove players that have already been seated from the pool
        for player in table_combs[best_table_ix]:
            temp_playerlist.remove(player)
    return output_tables